In [ ]:
import yfinance as yf
import pandas as pd

from pathlib import Path

In [ ]:
# 1. Parâmetros
ativo = "ITUB3.SA"
data_inicial = "2022-01-01"
data_final = "2025-12-31"

# 2. Download dos dados
tabela_cotacao = yf.download(
    ativo,
    start=data_inicial,
    end=data_final,
    progress=False,
    multi_level_index=False,
)

# 3. Cálculos de variação
tabela_cotacao["Variacao"] = tabela_cotacao["Close"].diff()
tabela_cotacao["Var%"] = tabela_cotacao["Close"].pct_change() * 100

# 4. Transformando o índice em datetime e garantindo que ele seja o Index
tabela_cotacao.index = pd.to_datetime(tabela_cotacao.index)
tabela_cotacao.columns.name = None

tabela_cotacao.head(10)

,Date,Close,High,Low,Open,Volume,Variacao,Var%
0,2022-01-03,12.387838,12.579601,12.170507,12.330310,1461343,NaN,NaN
1,2022-01-04,12.662697,12.688264,12.368661,12.368661,1115778,0.274858,2.218777
2,2022-01-05,12.407015,12.726618,12.387838,12.643520,984350,-0.255682,-2.019175
3,2022-01-06,12.630733,12.656301,12.368659,12.598774,2113838,0.223718,1.803155
4,2022-01-07,12.828888,12.848064,12.438971,12.681870,1116684,0.198155,1.568835
5,2022-01-10,12.899199,12.931161,12.688261,12.828888,964862,0.070312,0.548072
6,2022-01-11,13.084571,13.135708,12.860849,12.892809,1354841,0.185371,1.437077
7,2022-01-12,13.090965,13.206022,12.918379,13.084572,1132206,0.006394,0.048870
8,2022-01-13,13.257161,13.397786,13.020654,13.039830,1303063,0.166196,1.269546
9,2022-01-14,13.263553,13.365825,13.154887,13.180455,1009163,0.006392,0.048212


In [ ]:
caminho_raiz = Path.cwd().parent

pasta_parquet = caminho_raiz / "arquivos_parquet"
pasta_parquet.mkdir(parents=True, exist_ok=True)

caminho_arquivo = pasta_parquet / "dados_base.parquet"
tabela_cotacao.to_parquet(caminho_arquivo)

print(f"Arquivo salvo em: {caminho_arquivo}")

Arquivo salvo em: c:\Users\Pc Gamer\OneDrive\FIAP\Machine Learnig Engineering\Fase - 04\fiap_tech_challenge_fase4\arquivos_parquet\dados_base.parquet
